In [3]:
import pymongo
import re
import requests
import pandas as pd
import numpy as np
import urllib
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

In [4]:
client = pymongo.MongoClient('35.167.189.162', 27016)


In [5]:
db_ref = client.project4_database

In [6]:
coll_ref = db_ref.my_project4_combined_df

In [7]:
cursor = coll_ref.find_one()

cursor

sample_docs = (cursor)

In [8]:
from_mongo_combined = pd.DataFrame(sample_docs)

In [9]:
from_mongo_combined.shape

(2993, 5)

In [10]:
from_mongo_combined.head()

,_id,category,pageid,text,title
0,5a1a0a2db97736002e75b32c,Machine learning,43385931,data exploration is an approach similar to ini...,Data exploration
1,5a1a0a2db97736002e75b32c,Machine learning,49082762,these datasets are used for machinelearning re...,List of datasets for machine learning research
10,5a1a0a2db97736002e75b32c,Machine learning,10435998,katia sycara greek is a professor in the robot...,Katia Sycara
100,5a1a0a2db97736002e75b32c,Machine learning,871681,in statistics a mixture model is a probabilist...,Mixture model
1000,5a1a0a2db97736002e75b32c,Machine learning,20768311,pierre baldi is a chancellors professor of com...,Pierre Baldi


In [12]:
sample_30pct = from_mongo_combined.sample(900)

### Label Encode the Categories


In [14]:
from sklearn.preprocessing import LabelEncoder

In [15]:
le = LabelEncoder()
sample_30pct['category_numerical'] = le.fit_transform(sample_30pct['category'])

### Prepare Document Term Matrix

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
tfidf_vectorizer = TfidfVectorizer(min_df = 1, stop_words = 'english')

In [18]:
document_term_matrix_sps = tfidf_vectorizer.fit_transform(sample_30pct.text)

In [19]:
document_term_matrix_sps

<900x37806 sparse matrix of type '<class 'numpy.float64'>'
	with 202273 stored elements in Compressed Sparse Row format>

In [20]:
document_term_matrix_df = pd.DataFrame(document_term_matrix_sps.toarray(),
                                       index=sample_30pct.text,
                                       columns=tfidf_vectorizer.get_feature_names())

# Search

### Encode the search term using the same vectorizer

In [21]:
Machine_learning = [
{"sentence": "A Support Vector Machine (SVM) is a discriminative classifier", "title" : "Machine learning"},
{"sentence": "formally defined by a separating hyperplane.,", "title" : "Machine learning"},
{"sentence": "In other words, given labeled training data (supervised learning),", "title" : "Machine learning"},
{"sentence": "the algorithm outputs an optimal hyperplane ,", "title" : "Machine learning"},
{"sentence": "which categorizes new examples.", "title" : "Machine learning"},
{"sentence": "In which sense is the hyperplane obtained optimal?", "title" : "Machine learning"},
{"sentence": "Let’s consider the following simple problem:", "title" : "Machine learning"},
{"sentence": "For a linearly separable set of 2D-points", "title" : "Machine learning"},
{"sentence": "which belong to one of two classes, find a separating straight line.", "title" : "Machine learning"},
{"sentence": "In this example we deal with lines and points in the Cartesian plane,", "title" : "Machine learning"},
{"sentence": "instead of hyperplanes and vectors in a high dimensional space.", "title" : "Machine learning"},
{"sentence": "This is a simplification of the problem.", "title" : "Machine learning"},
{"sentence": "It is important to understand that this is done only because our intuition,", "title" : "Machine learning"},
{"sentence": "is better built from examples that are easy to imagine. ", "title" : "Machine learning"},
{"sentence": "Then, the operation of the SVM algorithm is based on finding the hyperplane", "title" : "Machine learnings"},
{"sentence": "that gives the largest minimum distance to the training examples.",    "title" : "Machine learning"},
]





In [22]:
machine_learning_df = pd.DataFrame(Machine_learning)

In [23]:
machine_learning_df.head()

,sentence,title
0,A Support Vector Machine (SVM) is a discrimina...,Machine learning
1,"formally defined by a separating hyperplane.,",Machine learning
2,"In other words, given labeled training data (s...",Machine learning
3,"the algorithm outputs an optimal hyperplane ,",Machine learning
4,which categorizes new examples.,Machine learning


In [24]:
machine_learning_df.sentence

0     A Support Vector Machine (SVM) is a discrimina...
1         formally defined by a separating hyperplane.,
2     In other words, given labeled training data (s...
3         the algorithm outputs an optimal hyperplane ,
4                       which categorizes new examples.
5     In which sense is the hyperplane obtained opti...
6          Let’s consider the following simple problem:
7             For a linearly separable set of 2D-points
8     which belong to one of two classes, find a sep...
9     In this example we deal with lines and points ...
10    instead of hyperplanes and vectors in a high d...
11             This is a simplification of the problem.
12    It is important to understand that this is don...
13    is better built from examples that are easy to...
14    Then, the operation of the SVM algorithm is ba...
15    that gives the largest minimum distance to the...
Name: sentence, dtype: object

In [25]:
search_terms_encoded = tfidf_vectorizer.transform(machine_learning_df.sentence)

In [26]:
search_terms_encoded_df = pd.DataFrame(search_terms_encoded.toarray(), 
                                       index=machine_learning_df.sentence, 
                                       columns=tfidf_vectorizer.get_feature_names())

### Select a Random Search Term

In [27]:
random_search_term_df = search_terms_encoded_df.sample()
random_search_term_df.head()

,aa,aaa,aaabc,aaabcaaabcaaabcaaabc,aaai,aaainumber,aaas,aabaabaa,aabar,aabsbaarightarrow,...,zvfz,zwart,zx,zxoxm,zxtimes,zxy,zymapsto,zynga,zyrion,zzero
sentence,,,,,,,,,,,,,,,,,,,,,
"It is important to understand that this is done only because our intuition,",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Append the search term to the document term matrix

In [28]:
dtm_with_search_term = document_term_matrix_df.append(random_search_term_df)

### Compute SVD of Augmented Document Term Matrix


In [30]:
from sklearn.decomposition import TruncatedSVD

In [31]:
n_components = 50
SVD = TruncatedSVD(n_components)
component_names = ["component_"+str(i+1) for i in range(n_components)]

In [32]:
svd_matrix = SVD.fit_transform(dtm_with_search_term)

In [33]:
svd_df = pd.DataFrame(svd_matrix, 
                      index=dtm_with_search_term.index, 
                      columns=component_names)

### Identify the Vector for our Search Term

In [34]:
search_term_svd_vector = svd_df.loc[random_search_term_df.index]
search_term_svd_vector[:5]

,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9,component_10,...,component_41,component_42,component_43,component_44,component_45,component_46,component_47,component_48,component_49,component_50
"It is important to understand that this is done only because our intuition,",0.008175,-0.002072,-0.004301,0.011646,-0.007777,-0.02235,0.001893,-0.005134,-0.000656,-0.007107,...,-0.005839,0.000588,0.003006,-0.003764,-0.025773,-0.008572,-0.01698,-0.01155,0.009429,-0.006736


### Use Cosine Similarity to Find the Most Similar Vector

In [35]:
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
svd_df['cosine_sim'] = cosine_similarity(svd_df, search_term_svd_vector)

In [37]:
svd_df[['cosine_sim']].sort_values('cosine_sim', ascending=False).head(5)

cosine_sim
It is important to understand that this is done...    1.000000
an erp system selection methodology is a formal...    0.498135
workforce optimization wfo is a business strate...    0.482582
in machine learning pattern recognition and in ...    0.459287
handwriting recognition or hwr is the ability o...    0.425101